In [17]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:

sample = """<h1>Title Goes Here</h1>
<b>Bolded Text</b>
<i>Italicized Text</i>
<img src="this should all be gone"/>
<a href="this will be gone, too">But this will still be here!</a>
I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?
[Some text we don't want to keep is in here]
¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!
something... is! wrong() with.,; this :: sentence.
I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?
My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.
Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.
[This is some other unwanted text]
John: "Well, well, well."
James: "There, there. There, there."
&nbsp;&nbsp;
There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.
I have to go get 2 tutus from 2 different stores, too.
22    45   1067   445
{{Here is some stuff inside of double curly braces.}}
{Here is more stuff in single curly braces.}
[DELETE]
</body>
</html>"""

In [19]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

sample = denoise_text(sample)
print(sample)

Title Goes Here
Bolded Text
Italicized Text

But this will still be here!
I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?

¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!
something... is! wrong() with.,; this :: sentence.
I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?
My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.
Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.

John: "Well, well, well."
James: "There, there. There, there."
  
There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.
I have to go get 2 tutus from 2 different stores, too.
22    45   1067   445
{{Here is some stuff inside of double curly braces.}}
{Here is m

In [20]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

sample = replace_contractions(sample)
print(sample)

Title Goes Here
Bolded Text
Italicized Text

But this will still be here!
I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?

¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!
something... is! wrong() with.,; this :: sentence.
I cannot do this anymore. I did not know them. Why could not you have dinner at the restaurant?
My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.
do not do it.... Just do not. Billy! I know what you are doing. This is a great little house you have got here.

John: "Well, well, well."
James: "There, there. There, there."
  
There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.
I have to go get 2 tutus from 2 different stores, too.
22    45   1067   445
{{Here is some stuff inside of double curly braces.}}
{H

In [21]:
words = nltk.word_tokenize(sample)
print(words)

[u'Title', u'Goes', u'Here', u'Bolded', u'Text', u'Italicized', u'Text', u'But', u'this', u'will', u'still', u'be', u'here', u'!', u'I', u'run', u'.', u'He', u'ran', u'.', u'She', u'is', u'running', u'.', u'Will', u'they', u'stop', u'running', u'?', u'I', u'talked', u'.', u'She', u'was', u'talking', u'.', u'They', u'talked', u'to', u'them', u'about', u'running', u'.', u'Who', u'ran', u'to', u'the', u'talking', u'runner', u'?', u'\xa1Sebasti\xe1n', u',', u'Nicol\xe1s', u',', u'Alejandro', u'and', u'J\xe9ronimo', u'are', u'going', u'to', u'the', u'store', u'tomorrow', u'morning', u'!', u'something', u'...', u'is', u'!', u'wrong', u'(', u')', u'with.', u',', u';', u'this', u':', u':', u'sentence', u'.', u'I', u'can', u'not', u'do', u'this', u'anymore', u'.', u'I', u'did', u'not', u'know', u'them', u'.', u'Why', u'could', u'not', u'you', u'have', u'dinner', u'at', u'the', u'restaurant', u'?', u'My', u'favorite', u'movie', u'franchises', u',', u'in', u'order', u':', u'Indiana', u'Jones', u'

In [22]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

words = normalize(words)
print(words)

[u'title', u'goes', u'bolded', u'text', u'italicized', u'text', u'still', u'run', u'ran', u'running', u'stop', u'running', u'talked', u'talking', u'talked', u'running', u'ran', u'talking', u'runner', u'sebastian', u'nicolas', u'alejandro', u'jeronimo', u'going', u'store', u'tomorrow', u'morning', u'something', u'wrong', u'sentence', u'anymore', u'know', u'could', u'dinner', u'restaurant', u'favorite', u'movie', u'franchises', u'order', u'indiana', u'jones', u'marvel', u'cinematic', u'universe', u'star', u'wars', u'back', u'future', u'harry', u'potter', u'billy', u'know', u'great', u'little', u'house', u'got', u'john', u'well', u'well', u'well', u'james', u'lot', u'reasons', u'one hundred and one', u'reasons', u'one million', u'reasons', u'actually', u'go', u'get', u'two', u'tutus', u'two', u'different', u'stores', u'twenty-two', u'forty-five', u'one thousand and sixty-seven', u'four hundred and forty-five', u'stuff', u'inside', u'double', u'curly', u'braces', u'stuff', u'single', u'cur

In [23]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

stems, lemmas = stem_and_lemmatize(words)
print('Stemmed:\n', stems)
print('\nLemmatized:\n', lemmas)

('Stemmed:\n', [u'titl', u'goe', u'bold', u'text', u'it', u'text', u'stil', u'run', u'ran', u'run', u'stop', u'run', u'talk', u'talk', u'talk', u'run', u'ran', u'talk', u'run', u'sebast', u'nicola', u'alejandro', u'jeronimo', u'going', u'stor', u'tomorrow', u'morn', u'someth', u'wrong', u'sent', u'anym', u'know', u'could', u'din', u'resta', u'favorit', u'movy', u'franch', u'ord', u'indian', u'jon', u'marvel', u'cinem', u'univers', u'star', u'war', u'back', u'fut', u'harry', u'pot', u'bil', u'know', u'gre', u'littl', u'hous', u'got', u'john', u'wel', u'wel', u'wel', u'jam', u'lot', u'reason', u'one hundred and on', u'reason', u'one million', u'reason', u'act', u'go', u'get', u'two', u'tut', u'two', u'diff', u'stor', u'twenty-two', u'forty-five', u'one thousand and sixty-seven', u'four hundred and forty-five', u'stuff', u'insid', u'doubl', u'cur', u'brac', u'stuff', u'singl', u'cur', u'brac'])
('\nLemmatized:\n', [u'title', u'go', u'bolded', u'text', u'italicize', u'text', u'still', u'ru

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)


NameError: name 'reviews_train_clean' is not defined